In [1]:
import os
import glob
import cv2
import matplotlib.pyplot as plt
import tifffile as tiff
import numpy as np

In [2]:
def enchancer_IMAGE(new_frame):
    #convert to 8bit image
    new_frame2 = cv2.normalize(new_frame, None, 255, 0, cv2.NORM_MINMAX, cv2.CV_8U)
    def gamma_trans(img, gamma):
        gamma_table=[np.power(x/255.0,gamma)*255.0 for x in range(256)]
        gamma_table=np.round(np.array(gamma_table)).astype(np.uint8)
        return cv2.LUT(img,gamma_table)

    image_gamma_correct=gamma_trans(new_frame2,0.5)

    alpha = 0.5
    beta = 100
    image_gamma_correct=cv2.convertScaleAbs(image_gamma_correct, alpha=alpha, beta=beta)
    return image_gamma_correct

In [3]:
#subfolder path
path = r''
path=path+'\\'

In [5]:
#calibration -- mkm in pixel
_10_mk_in_pixel = 110

In [6]:
for sub_folders in os.scandir(path):
    if sub_folders.is_dir():
        name=sub_folders.path.split('\\')[-1]
        files = glob.glob(path+name+'\\'+'*.tif')
        
        #create a subfolder for images with bars.
        name_folder_with_bars = 'with_bars_'+name
        name_folder_with_bars = path +'!Figs_with_bars'+'\\'+name_folder_with_bars
        os.makedirs(name_folder_with_bars, exist_ok=True)
        for i in files:
            name_fig=i.split('\\')[-1]

            img = tiff.imread(i)
            
            kl=enchancer_IMAGE(img)
            height, width = kl.shape
            
            lineThickness=10
            cv2.line(kl, (int(width*0.90-_10_mk_in_pixel/2),int(height*0.95)), (int(width*0.90+_10_mk_in_pixel/2),int(height*0.95)), (255,255,255), lineThickness)
            
            color = 'w'
            fig, ax = plt.subplots()
            ax.imshow(kl, cmap = 'gray')

            #m=plt.text(0.85, 0.1, r'100 $\mu$m', transform=ax.transAxes)
            m=plt.text(int(width*0.85), int(height*0.9), r'10 $\mu$m')
            m.set_bbox(dict(facecolor=color, alpha=0.5))
            plt.axis('off')
            
            fig.savefig(name_folder_with_bars+'\\'+name_fig.split('.')[0] + '.png', dpi = 300, format='png',bbox_inches='tight')
            
            plt.close(fig)
